In [1]:
from PIL import Image

import numpy as np

from skimage.restoration import inpaint

def load_image(image_path):

    img = Image.open(image_path)

    return img

def mark_watermark_area(img, x1, y1, x2, y2):

    img_array = np.array(img)

    mask = np.zeros(img_array.shape[:2], dtype=np.uint8)

    mask[y1:y2, x1:x2] = 1

    return img_array, mask

def remove_watermark(img_array, mask):

    result = inpaint.inpaint_biharmonic(img_array, mask, multichannel=True)

    return result

def save_image(result, output_path):

    img_result = Image.fromarray(result.astype('uint8'))

    img_result.save(output_path)

# 主程序

image_path = 'path/to/your/image.jpg'

output_path = 'path/to/output/image.jpg'

x1, y1, x2, y2 = 100, 100, 200, 200

img = load_image(image_path)

img_array, mask = mark_watermark_area(img, x1, y1, x2, y2)

result = remove_watermark(img_array, mask)

save_image(result, output_path)

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/image.jpg'